In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME, params={"name": f"{str(uuid.uuid4())[:8]}-task", "params": {}}
)

task = rpc_response.result
print(task)

Task(id='e5333f10-5fe2-4862-8c89-7b422e27a471', created_at=datetime.datetime(2025, 10, 2, 0, 17, 9, 695914, tzinfo=TzInfo(UTC)), name='ffba53be-task', params={}, status='RUNNING', status_reason='Task created, forwarding to ACP server', task_metadata=None, updated_at=datetime.datetime(2025, 10, 2, 0, 17, 9, 695914, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Tell me about recent AI news for today only."},
        "task_id": task.id,
    },
)

event = rpc_response.result
print(event)

Event(id='76ab0b9a-b107-4199-a5f6-31315dc43ee2', agent_id='2f4d3b3d-6a59-46ff-993e-afd6c1f1c7ab', sequence_id=131, task_id='e5333f10-5fe2-4862-8c89-7b422e27a471', content=TextContent(author='user', content='Tell me about recent AI news for today only.', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 10, 2, 0, 17, 9, 776113, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task,
    only_after_timestamp=event.created_at,
    print_messages=True,
    rich_print=True,
    timeout=120,
)

╭───────────────────────── USER [10/02/2025 00:17:09] ─────────────────────────╮
│ Tell me about recent AI news for today only.                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:17:13] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Searching for AI news                                                        │
│                                                                              │
│ The user wants an update on recent AI news specifically for today, October   │
│ 2, 2025. I’ll use the web search tool to find this information quickly,      │
│ aiming for outlets that are major or well-known. I plan to use a query like  │
│ "AI news October 2 2025" or "AI news today October 2 2025". It’ll help to    │
│ set the search context size high to ensure thorough coverage on this topic.  │
│ Let's go ahead and get this done!                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:17:16] ─────────────────────────╮
│ 🔧 Tool Request: openai_web_search                                           │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "arguments": "{\"input\":\"AI news October 2 2025 or 'today' Oct 2 2025   │
│  major AI                                                                    │
│  announcements\",\"model\":\"gpt-5-mini\",\"reasoning_effort\":\"low\",\"ty  │
│  \":\"web_search_preview\",\"search_context_size\":\"high\"}",               │
│    "call_id": "call_tKw9WT0BYQyCqEcgJ2rNjoy3",                               │
│    "name": "openai_web_search",                                              │
│    "type": "function_call",                                                  │
│    "id": "fc_021a54d0dc6d53340068ddc48d04c481a393f03e37e300827e",            │
│    "status": "completed"                                                     │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:17:34] ─────────────────────────╮
│ ✅ Tool Response: openai_web_search                                          │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Here are the major AI-related announcements and headlines aroun  │
│  today (October 2, 2025). I focused on the biggest product, infrastructure,  │
│  and partnership news; sources are listed after each item.\n\n- Microsoft    │
│  launches Microsoft 365 Premium (Copilot-integrated) for individuals \u2014  │
│  new $19.99/month plan that bundles Copilot across Outlook/Word/Excel, rais  │
│  Copilot usage limits for some existing Personal/Family customers, and       │
│  replaces Copilot Pro. (Reported Oct 1\u20132, 2025).                        │
│  ([reuters.com](https://www.reuters.com/technology/microsoft-launches-ai-po  │
│  red-365-premium-bundle-1999-per-month-2025-10-01/?utm_source=openai))\n\n-  │
│  OpenAI announces expanded Stargate partnerships in South Korea with Samsun  │
│  and SK (memory and data\u2011center collaboration) as part of its large     │
│  \u201cStargate\u201d infrastructure push and related global datacenter      │
│  plans. (Reported Oct 2, 2025).                                              │
│  ([apnews.com](https://apnews.com/article/a65fd1a21a8587c991cc30b94b1dfe89?  │
│  m_source=openai))\n\n- Google launches new Nest cameras and a redesigned    │
│  Google Home app built for Gemini for Home, introducing Gemini-integrated    │
│  home features (descriptive alerts, \u201cHome Brief,\u201d Ask Home chatbo  │
│  and new subscription tiers. (Reported Oct 1\u20132, 2025).                  │
│  ([theverge.com](https://www.theverge.com/news/789412/new-nest-cams-nest-do  │
│  bell-launch-price-specs-release-date?utm_source=openai))\n\n- NVIDIA        │
│  continues major infrastructure/product rollouts: recent announcements arou  │
│  Rubin CPX (GPU class for massive-context inference), Dynamo                 │
│  inference-serving software, Blackwell Ultra/AI Factory platform, and        │
│  availability plans for AI foundation models on RTX AI PCs \u2014 signaling  │
│  heavy pushes on inference scale, large-context models, and on\u2011device   │
│  for creators/enterprises. (Company releases Sept\u2013Oct 2025).            │
│  ([investor.nvidia.com](https://investor.nvidia.com/news/press-release-deta  │
│  s/2025/NVIDIA-Unveils-Rubin-CPX-A-New-Class-of-GPU-Designed-for-Massive-Co  │
│  ext-Inference/default.aspx?utm_source=openai))\n\n- Industry events /       │
│  company summits with product demos and roadmaps: OpenAI DevDay scheduled O  │
│  6, 2025 (preview teasers and DevDay announcements expected), and Anthropic  │
│  held a London Builder Summit Oct 1 with demos of Claude and discussions of  │
│  autonomous-agent work. These events are driving near-term product and       │
│  developer announcements.                                                    │
│  ([openai.com](https://openai.com/index/announcing-devday-2025/?utm_source=  │
│  enai))\n\nIf you want, I can:\n- Pull full articles and timelines for any   │
│  the items above (e.g., full Reuters/AP/Verge/NVIDIA coverage).\n- Summariz  │
│  the expected user impact, pricing, or migration steps (e.g., what Microsof  │
│  365 Premium means if you\u2019re a Copilot Pro or Personal subscriber).\n-  │
│  Track other outlets for any breaking follow-ups today (Oct 2,               │
│  2025).\n\nWhich of these would you like more detail on?",                   │
│    "annotations": null,       

╭──────────────────────── AGENT [10/02/2025 00:17:37] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Ensuring news accuracy                                                       │
│                                                                              │
│ I found a summary that included items from both October 1 and 2, but since   │
│ the user specifically asked for news from October 2, I'm needing to clarify  │
│ that. The summary flagged items related to OpenAI partnerships specifically  │
│ from October 2, while Microsoft and Google Nest news fell within the range   │
│ of October 1–2. To refine this, I should run a new search focused on         │
│ "October 2, 2025 AI news" for accuracy. Let’s go for high-context results!   │
╰──────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────── AGENT [10/02/2025 00:19:23] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Gathering AI news summaries                                                  │
│                                                                              │
│ I noticed that some items are from earlier dates but were listed as closing  │
│ on October 2. The user probably wants a brief list of the top AI news items  │
│ reported today. I should create concise bullet points with sources and 1-2   │
│ sentence summaries. It might be a good idea to offer to expand on any item   │
│ or provide full article links. Plus, I’ll mention that coverage evolves and  │
│ offer to keep monitoring for updates.                                        │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:19:26] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing AI stories for today                                             │
│                                                                              │
│ I’m crafting the final answer by starting with, "Here are the main AI        │
│ stories reported today (Oct 2, 2025):" First, Meta plans to use AI-chatbot   │
│ conversations to target ads, starting December 16, with notifications on     │
│ October 7, although it excludes the UK, EU, and South Korea initially and    │
│ sensitive topics.                                                            │
│                                                                              │
│ Next, there’s news about OpenAI expanding partnerships in South Korea with   │
│ Samsung and SK, and launching the Sora short-video app. Finally, although    │
│ the user wants "today only," I might mention the EU consultation on          │
│ transparency is relevant too.                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:19:28] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Crafting the final message                                                   │
│                                                                              │
│ I could ask the user if they want the full articles in addition to the       │
│ summaries. I need to keep things concise and make direct offers like, "Do    │
│ you want full articles, a deeper summary, or tracking updates?" It’s         │
│ important to mention sources as well, and since the search provided URLs in  │
│ the tool outputs, I can include those in parentheses. Alright, I’ll put all  │
│ this together to create the final message!                                   │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:19:31] ─────────────────────────╮
│ Here are the main AI stories reported today (October 2, 2025):               │
│                                                                              │
│  • Meta will begin using conversations with its Meta AI assistant to         │
│    personalize ads and content (WSJ / Reuters). Key points: Meta says it     │
│    will add signals from AI chats to ad/content personalization (excluding   │
│    certain “sensitive” topics), notify users starting Oct 7, and the change  │
│    takes effect Dec 16; rollout initially excludes the U.K., EU and South    │
│    Korea. Sources: Wall Street Journal (Oct 2) and Reuters coverage.         │
│    Links:                                                                    │
│    https://www.wsj.com/tech/ai/meta-will-begin-using-ai-chatbot-conversation │
│    s-to-target-ads-291093d3 and                                              │
│    https://www.reuters.com/business/media-telecom/meta-use-ai-chats-personal │
│    ize-content-ads-december-2025-10-01/                                      │
│  • OpenAI expands its “Stargate” infrastructure partnerships in South Korea  │
│    (AP / Reuters). OpenAI signed MOUs/LOIs with Samsung and SK Group to      │
│    secure advanced memory supply (HBM/DRAM) and explore data‑center          │
│    collaboration as part of its global Stargate build‑out. Coverage          │
│    highlights meetings in Seoul between OpenAI leadership and South Korean   │
│    officials/companies. Source: AP News (Oct 2) and Reuters.                 │
│    Link: https://apnews.com/article/a65fd1a21a8587c991cc30b94b1dfe89         │
│  • OpenAI launches Sora (short-video / generative-video app) and related     │
│    generative-video features; ChatGPT shopping/Instant Checkout pilots       │
│    continue (tech press reports). Outlets report OpenAI rolling out an       │
│    invite-style short-video app (Sora) built on its video-generation models  │
│    and expanding ChatGPT shopping integrations (initial merchants include    │
│    Etsy sellers). Sources: major tech outlets reporting Oct 1–2 (examples:   │
│    MacRumors, CNBC/tech sites).                                              │
│    Example link:                                                             │
│    https://www.macrumors.com/2025/09/30/openai-sora-ai-video-app/            │
│  • AI governance/regulatory items tied to Oct 2: EU transparency             │
│    consultation on AI‑generated content and labeling closes today (feeds     │
│    ongoing EU transparency/labeling work), and the U.S. press continues      │
│    debate about private-sector AI oversight initiatives. Sources include EU  │
│    filings and Politico / other regulatory coverage.                         │
│                                                                              │
│ Would you like me to:                                                        │
│                                                                              │
│  • Pull the full articles for any of the items above and summarize them in   │
│    more detail?                                                              │
│  • Focus on one topic (privacy implications for Meta’s change,               │
│    technical/market impact of OpenAI’s Stargate deals, how Sora works and    │
│    availability)?                                                            │
│  • Monitor the rest of today for any breaking updates (I can check again and │
│    send any new items)?                                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 120 seconds - returning collected messages
